<a href="https://colab.research.google.com/github/joshfpedro/math-328/blob/main/least_squares_approximation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Generate noisy data
np.random.seed(42)
x_data = np.linspace(0, 10, 20)
y_true_data = 2 + 0.5 * x_data - 0.05 * x_data**2
y_noisy = y_true_data + np.random.normal(0, 0.5, len(x_data))

# Interpolation (goes through all points)
x_fine = np.linspace(0, 10, 300)
y_interp = lagrange_interpolation(x_data, y_noisy, x_fine)

# Least squares approximation (degree 2)
coeffs = np.polyfit(x_data, y_noisy, 2)
y_approx = np.polyval(coeffs, x_fine)

# True function on fine grid
y_true = 2 + 0.5 * x_fine - 0.05 * x_fine**2

plt.figure(figsize=(10, 5))
plt.plot(x_data, y_noisy, 'ro', markersize=8, label='Noisy data', alpha=0.6)
plt.plot(x_fine, y_true, 'g-', linewidth=3, label='True function', alpha=0.5)
plt.plot(x_fine, y_interp, 'b-', linewidth=1.5, label='Lagrange interpolation', alpha=0.7)
plt.plot(x_fine, y_approx, 'orange', linewidth=2.5, label='Least squares (degree 2)')
plt.xlabel('x', fontsize=12)
plt.ylabel('y', fontsize=12)
plt.title('Interpolation vs Least Squares Approximation', fontsize=14, fontweight='bold')
plt.grid(True, alpha=0.3)
plt.legend(fontsize=10)
plt.tight_layout()
plt.savefig('figures/interpolation_vs_approximation.png', dpi=300, bbox_inches='tight')
plt.close()